In [1]:
# Import libraries
from gurobipy import*

import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
# File path 
FILE = "./DaSilvaUrrutia/n200w100.001.txt"

In [3]:
# Default params
SUPPORTED_FORMAT = ['NUM', 'X', 'Y', 'DEMAND', 'READYTIME', 'DUEDATE', 'SERVICE']
MINUTES = 60
HOURS = 3600
OFFSET_TIMES = 8*HOURS

COLUMNS_OPS = {'NUM': lambda x: float(x),
               'X': lambda x: float(x),
               'Y': lambda x: float(x),
               'DEMAND': lambda x: 1,
               #'READYTIME': lambda x: float(x)+OFFSET_TIMES,
               'READYTIME': lambda x: float(x),
               #'DUEDATE': lambda x: float(x)+OFFSET_TIMES,
               'DUEDATE': lambda x: float(x),

               'SERVICE': lambda x: 30*MINUTES
              }

AGENTS = 10

TIME_PER_DISTANCE = 1

#WORKING_TIME_RANGE = (OFFSET_TIMES, OFFSET_TIMES + 8*HOURS)
WORKING_TIME_RANGE = (0, 8*HOURS)

#LUNCH_BREAK_RANGE = (12*HOURS, 13.5*HOURS)
LUNCH_BREAK_RANGE = (12*HOURS-OFFSET_TIMES, 13.5*HOURS-OFFSET_TIMES)
LUNCH_BREAK_TIME = 30*MINUTES

OFFICE_NUM = 0
OFFICE_X = .0
OFFICE_Y = .0
OFFICE_READYTIME = WORKING_TIME_RANGE[0]
OFFICE_DUEDATE = WORKING_TIME_RANGE[1]
OFFICE_SERVICE = 1*HOURS

In [4]:
LUNCH_BREAK_RANGE

(14400, 19800.0)

In [5]:
WORKING_TIME_RANGE

(0, 28800)

In [6]:
def read_input_tsptw(filename):
    """This function is used to convert input file to usable data"""
    nb_nodes = 0
    
    data_dict = dict()
    
    nodes_x = list()
    nodes_y = list()
       
    # Open file and read lines 
    with open(filename, "r") as file:
        # Initialize columns in empty dict
        columns = file.readline().replace("#","").split()
        if columns != SUPPORTED_FORMAT:
            print("ERROR! Format not supported.")
            return 
        
        # Add office to data
        data_dict.update({OFFICE_NUM: {'X': OFFICE_X, 
                                       'Y': OFFICE_Y, 
                                       'DEMAND': AGENTS,
                                       'READYTIME': OFFICE_READYTIME,
                                       'DUEDATE': OFFICE_DUEDATE,
                                       'SERVICE': OFFICE_SERVICE,}})
        # Add office to nodes
        nodes_x.append(OFFICE_X)
        nodes_y.append(OFFICE_Y)
            
        # For each data line
        for line in file.readlines():
            node_dict = {k: COLUMNS_OPS[k](val) for k, val in zip(columns, line.split())}
                
            # Get id
            node_id = node_dict.pop('NUM')
            # Insert new node in data dict
            data_dict.update({int(node_id): node_dict})            
            # Get nodes positions
            nodes_x.append(float(line.split()[columns.index('X')]))
            nodes_y.append(float(line.split()[columns.index('Y')]))

    # Get distance matrix
    distance_matrix = compute_distance_matrix(nodes_x, nodes_y)
    
    return (data_dict, distance_matrix, dict(enumerate(zip(nodes_x, nodes_y))))


def compute_distance_matrix(nodes_x, nodes_y):
    """This function is used to compute the distance matrix"""
    nb_customers = len(nodes_x)
    distance_matrix = [[None for i in range(nb_customers)] for j in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(nodes_x[i], nodes_x[j], nodes_y[i], nodes_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


def compute_dist(xi, xj, yi, yj):
    """This function is used to compute euclidean distance"""
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5)) * TIME_PER_DISTANCE

In [7]:
# Getting parameters
data_dict, distance_matrix, positions = read_input_tsptw(FILE)

In [8]:
# DEBUG RESTRICTIONS
C = 20
data_dict = {k: v for k,v in data_dict.items() if k < C}
distance_matrix = distance_matrix[:C][:C]


In [9]:
# Create model
mod = Model("TSPTW")    


--------------------------------------------
--------------------------------------------

Using license file c:\gurobi903\gurobi.lic
Academic license - for non-commercial use only


In [10]:
pos = list(range(C))

In [11]:
# VARIABLES
# Served client
x = mod.addVars({(i,a): 0 for i in pos
                          for a in range(AGENTS)},
               name="x",
               vtype=GRB.BINARY)

#  Agent trip
y = mod.addVars({(i,j,a): 0 for i in pos
                            for j in pos 
                            for a in range(AGENTS)}, 
                name="y", 
                vtype=GRB.BINARY)

# Time spent at Client / Office in specific trip
c = mod.addVars({(i,j,a): 0 for i in pos
                            for j in pos
                            for a in range(AGENTS)}, 
                name="c", 
                vtype=GRB.INTEGER)

# Serve Order
o = mod.addVars({(i,a): 0 for i in pos
                          for a in range(AGENTS)},
               name="o",
               vtype=GRB.INTEGER)

# Serve time
s =  mod.addVars({(i,a): 0 for i in pos 
                           for a in range(AGENTS)}, 
                 name="s", 
                 vtype=GRB.INTEGER)

# Wait time
w = mod.addVars({(i,a): 0 for i in pos
                          for a in range(AGENTS)}, 
                name="w", 
                vtype=GRB.INTEGER)

# Lunch done between customers
l = mod.addVars({(i,j,a): 0 for i in pos
                            for j in pos
                            for a in range(AGENTS)}, 
                name="l", 
                vtype=GRB.INTEGER)

# Is lunch to do
t = mod.addVars({(a): 0 for a in range(AGENTS)},
               name="t",
               vtype=GRB.INTEGER)

In [12]:
# CONSTRAINTS

# All client must be visited from an Agent
_= mod.addConstrs((quicksum(x[i,a]
                            for a in range(AGENTS)) == 1
                  for i in pos),
                  name="ServeAll")

# Agent visit Client once and only if he serve him
_= mod.addConstrs((quicksum(y[i,j,a]
                            for i in pos if i != j
                           ) <= x[j,a] 
                  for a in range(AGENTS)
                  for j in pos),
                  name="ServeOnceA")

# Agent start it's trip from a Client served by him or from the Office
_= mod.addConstrs((quicksum(y[i,j,a] 
                            for j in pos if i != j
                           ) <= x[i,a]
                  for a in range(AGENTS)
                  for i in pos),
                  name="ServeOnceB")

# _= mod.addConstrs((y[i,j,a]+y[j,i,a] <= 1 
#                   for i in pos
#                   for j in pos if i!=j
#                   for a in range(AGENTS)),
#                   name="NoTwoNodesLoops")

# Agent can't do loop between same Client
_= mod.addConstrs((y[i,i,a] == 0
                  for a in range(AGENTS)
                  for i in pos),
                  name="NoSelfLoops")

# # Sum of time spent into a position in a specific trip
# _= mod.addConstrs((quicksum(c[i,j,a]*y[i,j,a] for i in pos) == x[j,a] * data_dict[j]['SERVICE']
#                   for j in pos
#                   for a in range(AGENTS)),
#                   name="ServingTime"
#                  )

# # Can spent time in a position only if it's in the trip
# _= mod.addConstrs((c[i,j,a] <= y[i,j,a] * data_dict[j]['SERVICE'] 
#                    for i in pos
#                    for j in pos
#                    for a in range(AGENTS)),
#                   name="ServingOnlyIfInTrip"
#                  )

# _= mod.addConstrs((quicksum(y[i,j,a]+y[j,i,a]
#                             for i in pos if i != j
#                            ) <= 2*x[j,a] 
#                   for a in range(AGENTS)
#                   for j in pos),
#                   name="GoOnlyToServed")

# _= mod.addConstrs((quicksum(y[i,j,a]+y[j,i,a]
#                             for i in pos if i != j
#                            ) >= x[j,a] 
#                   for a in range(AGENTS)
#                   for j in pos),
#                   name="ServeOnce")

# _= mod.addConstrs((quicksum(y[i,j,a] 
#                             for i in pos
#                             for j in pos
#                            ) == 
#                    quicksum(x[i,a] for i in pos)-1
#                    for a in range(AGENTS)),
#                   name="EdgeCount")

# _= mod.addConstrs((quicksum([quicksum(y[i,h,a] for i in pos) * quicksum(y[h,j,a] for j in pos) 
#                        for h in pos]) ==
#                    quicksum(x[i,a] for i in pos)-2
#                    for a in range(AGENTS)),
#                   name="EachClientHasOneBefore")

# _= mod.addConstrs((quicksum([(1-quicksum(y[i,h,a] for i in pos)) * quicksum(y[h,j,a] for j in pos) 
#                        for h in pos]) == 1
#                    for a in range(AGENTS)),
#                   name="OneStart")

# _= mod.addConstrs((quicksum([quicksum(y[i,h,a] for i in pos) * (1-quicksum(y[h,j,a] for j in pos))
#                        for h in pos]) == 1
#                    for a in range(AGENTS)),
#                   name="OneEnd")

# se arco ij è isolato allora 1
# il numero di nodi isolati può essere al massimo 2
# _= mod.addConstrs(((1-quicksum(y[h,i,a] for h in pos)) +
#                    y[i,j,a] +
#                    (1-quicksum(y[j,k,a] for k in pos)) 
                   
#                        for i in pos
#                        for j in pos
#                    for a in range(AGENTS)),
#                   name="SinglePath")

# Serving order for each Agent served Client is calculated as 
# (is actual Client served) + (serving order of Client before him)
_= mod.addConstrs((o[i,a] == x[i,a]+quicksum(o[j,a]*y[j,i,a] for j in pos if i!=j) 
                  for a in range(AGENTS)
                  for i in pos),
                  name="ServeOrder")

# Serving order check is calculated as:
# (sum of all serving values) must be equal to (served Clients)*(served Clients + 1) / 2
_= mod.addConstrs((quicksum(o[i,a] for i in pos) == quicksum(x[i,a] for i in pos)*(quicksum(x[i,a] for i in pos)+1)/2
                  for a in range(AGENTS)),
                  name="CheckServeOrder")

# Sum of minutes spent in: travels, servicing clients, waiting, eating at lunch 
_= mod.addConstrs((quicksum(y[i,j,a] * distance_matrix[i][j]
                           for i in pos
                           for j in pos) +
                   quicksum(x[i,a] * data_dict[i]['SERVICE']
                           for i in pos) +
                   quicksum(w[i,a]
                           for i in pos) +
                   WORKING_TIME_RANGE[0] + 
                   quicksum(l[i,j,a]
                            for i in pos
                            for j in pos) * LUNCH_BREAK_TIME <= WORKING_TIME_RANGE[1]
                   for a in range(AGENTS)),
                  name="MaxHours")

In [13]:
# Service time of Client J is equal to sum of:
# wait_time_of_client_J, service_time_of_client_I_before_of_J, trip_between_I_and_J, 
# time_of_service_of_client_I, lunch_time_if_present
_= mod.addConstrs((s[j,a] == quicksum(y[i,j,a]*s[i,a] + 
                                      y[i,j,a]*distance_matrix[i][j] +
                                      y[i,j,a]*data_dict[i]['SERVICE'] +
                                      y[i,j,a]*l[i,j,a]*LUNCH_BREAK_TIME
                                      for i in pos if i!=j)
                                    +x[j,a]*(w[j,a])
                    for a in range(AGENTS)
                    for j in pos), 
                  name="RouteInTime")

# Agent serve Client after his time window start
_= mod.addConstrs((data_dict[i]['READYTIME']*x[i,a] <= s[i,a]
                   for i in pos
                   for a in range(AGENTS)),
                  name="ServeAfterTWStart")

# Agent serve Client before his time window end
_= mod.addConstrs((data_dict[i]['DUEDATE']*x[i,a] >= s[i,a]
                   for i in pos
                   for a in range(AGENTS)),
                  name="ServeBeforeTWEnd")

# If Agent have lunch between Clients I and J, agent have to make trip between I and J  
_= mod.addConstrs((l[i,j,a] <= y[i,j,a]
                   for i in pos
                   for j in pos
                   for a in range(AGENTS)),
                  name="LunchTime")

# If Agent has lunch it must be after lunch time start
_= mod.addConstrs((l[i,j,a]*(s[i,a] + 
                             data_dict[i]['SERVICE'] + 
                             distance_matrix[i][j] + 
                             w[j,a]) >= l[i,j,a]*LUNCH_BREAK_RANGE[0] 
                   for i in pos
                   for j in pos
                   for a in range(AGENTS)),
                  name="LunchTimeStart")

# If Agent has lunch it must be before lunch time end
_= mod.addConstrs((l[i,j,a]*(s[i,a] + 
                             data_dict[i]['SERVICE'] +
                             distance_matrix[i][j] +
                             w[j,a]) <= l[i,j,a]*LUNCH_BREAK_RANGE[1] 
                   for i in pos
                   for j in pos
                   for a in range(AGENTS)),
                  name="LunchTimeEnd")

# Getting maximum Agent service time
_= mod.addConstrs((t[a] == max_(s[i,a] for i in pos) 
                   for a in range(AGENTS)),
                  name="MaxServiceTime")

# If Agent working time is greater than lunch time than Agent must have lunch
_= mod.addConstrs((WORKING_TIME_RANGE[1] * (quicksum(l[i,j,a]
                                             for i in pos
                                             for j in pos)) >= t[a] - LUNCH_BREAK_RANGE[0]
                   for a in range(AGENTS)),
                  name="AgentNeedLunchI")

# If Agent working time is lesser than lunch time than Agent must not have lunch
_= mod.addConstrs((WORKING_TIME_RANGE[1] * (1 - quicksum(l[i,j,a]
                                             for i in pos
                                             for j in pos)) >= LUNCH_BREAK_RANGE[0] - t[a]
                   for a in range(AGENTS)),
                  name="AgentNeedLunchII")

# 
# _= mod.addConstrs((w[j,a] == quicksum(y[i,j,a]*
#                                       (data_dict[j]['READYTIME'] - 
#                                        s[i,a] -
#                                        distance_matrix[i][j])
#                                       for i in pos
#                                      )
#                    for j in pos
#                    for a in range(AGENTS)),
#                   name="WaitForClient")

In [14]:
mod.setObjective(quicksum(y[i,j,a]*distance_matrix[i][j]
                  for i in pos
                  for j in pos
                  for a in range(AGENTS)) + 
                 quicksum(w[i,a]
                  for i in pos
                  for a in range(AGENTS)), 
                 GRB.MINIMIZE)

In [15]:
# mod.params.Method=0
mod.params.TimeLimit=100
mod.optimize()

Changed value of parameter TimeLimit to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5050 rows, 12810 columns and 33400 nonzeros
Model fingerprint: 0x9e69d159
Model has 8410 quadratic constraints
Model has 10 general constraints
Variable types: 0 continuous, 12810 integer (4200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  QMatrix range    [5e-01, 2e+03]
  QLMatrix range   [5e-01, 2e+04]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Presolve removed 2210 rows and 7900 columns
Presolve time: 0.19s
Presolved: 41750 rows, 16510 columns, 120580 nonzeros
Presolved model has 200 SOS constraint(s)
Variable types: 0 continuous, 16510 integer (9000 binary)

Root relaxation: objective 1.084971e+04, 6805 iterations, 0.67 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [16]:
from copy import deepcopy
def domino(otrip):
    trip = deepcopy(otrip)
    sorted_trip = list()
    while len(trip):
        old = deepcopy(trip)
        # Find start
        for x in trip:
            not_start = False
            for y in trip:
                if x[0] == y[1]:
                    not_start = True
                    break
            if not not_start:
                sorted_trip.append(trip.pop(trip.index(x)))
                break
        # Continue dominos trip
        next_found = True
        while next_found:
            next_found = False
            for x in trip:
                if sorted_trip[-1][1] == x[0]:
                    sorted_trip.append(trip.pop(trip.index(x)))
                    next_found = True
                    break
                    
        if trip == old:
            for t in trip:
                sorted_trip.append(trip.pop(trip.index(x)))
    return sorted_trip

for a in range(AGENTS):
    agent_trip = list()
    for i in pos:
        for j in pos:
            if y[i,j,a].X:
                agent_trip.append((i,j))
                
    # Find start
    print(f"Agent {a}: {agent_trip}")

# PERCORSI DI VISITA

Agent 0: []
Agent 1: []
Agent 2: []
Agent 3: []
Agent 4: [(12, 4)]
Agent 5: [(1, 3), (11, 17), (17, 1)]
Agent 6: []
Agent 7: [(0, 8), (16, 0), (18, 16)]
Agent 8: []
Agent 9: [(6, 14), (15, 19), (19, 6)]


In [17]:
# ORDINE DI VISITA (ordine di visita, cliente visitato)
for a in range(AGENTS):
    agent_clients = list()
    for i in pos:
        if o[i,a].X:
            agent_clients.append((o[i,a].X, i))
    # Find start
    print(f"Agent {a}: {sorted(agent_clients, key=lambda tup: tup[0])}")

Agent 0: [(1.0, 9)]
Agent 1: [(1.0, 7)]
Agent 2: [(1.0, 10)]
Agent 3: [(1.0, 2)]
Agent 4: [(1.0, 12), (2.0, 4)]
Agent 5: [(1.0, 11), (2.0, 17), (3.0, 1), (4.0, 3)]
Agent 6: [(1.0, 5)]
Agent 7: [(1.0, 18), (2.0, 16), (3.0, 0), (4.0, 8)]
Agent 8: [(1.0, 13)]
Agent 9: [(1.0, 15), (2.0, 19), (3.0, 6), (4.0, 14)]


In [18]:
# MAX TEMPO VISITA
for a in range(AGENTS):
    # Find start
    print(f"Agent {a}: {t[a].X}")

Agent 0: 198.0
Agent 1: 100.0
Agent 2: 240.0
Agent 3: 6.0
Agent 4: 4002.0
Agent 5: 9046.0
Agent 6: 472.0
Agent 7: 10077.0
Agent 8: 351.0
Agent 9: 9740.0


In [19]:
# CHE CLIENTI VISITA OGNI AGENTE
for a in range(AGENTS):
    agent_clients = list()
    for i in pos:
        if x[i,a].X:
            agent_clients.append(i)
    # Find start
    print(f"Agent {a}: {agent_clients}")

Agent 0: [9]
Agent 1: [7]
Agent 2: [10]
Agent 3: [2]
Agent 4: [4, 12]
Agent 5: [1, 3, 11, 17]
Agent 6: [5]
Agent 7: [0, 8, 16, 18]
Agent 8: [13]
Agent 9: [6, 14, 15, 19]


In [20]:
# ATTESE DI VISITA E TEMPI DI VISITA
for a in range(AGENTS):
    agent_clients = list()
    for i in pos:
        if w[i,a].X:
            agent_clients.append((i, w[i,a].X, s[i,a].X, data_dict[i]["READYTIME"]))
    # Find start
    print(f"Agent {a}: {agent_clients}")

Agent 0: [(9, 198.0, 198.0, 198.0)]
Agent 1: [(7, 100.0, 100.0, 100.0)]
Agent 2: [(10, 240.0, 240.0, 240.0)]
Agent 3: [(2, 6.0, 6.0, 6.0)]
Agent 4: [(4, 1630.0, 4002.0, 4002.0), (12, 526.0, 526.0, 526.0)]
Agent 5: [(1, 1052.0, 7196.0, 0.0), (11, 939.0, 939.0, 904.0), (17, 1532.0, 4312.0, 4312.0)]
Agent 6: [(5, 472.0, 472.0, 472.0)]
Agent 7: [(8, 141.0, 10077.0, 10077.0), (16, 1074.0, 4445.0, 4445.0), (18, 1523.0, 1523.0, 1523.0)]
Agent 8: [(13, 351.0, 351.0, 351.0)]
Agent 9: [(6, 54.0, 6189.0, 6189.0), (14, 1696.0, 9740.0, 9740.0), (15, 779.0, 779.0, 729.0), (19, 1677.0, 4319.0, 4319.0)]


In [21]:
# ATTESE DI VISITA E TEMPI DI VISITA
for a in range(AGENTS):
    agent_clients = list()
    if t[a].X:
        # Find start
        print(f"Agent {a} have lunch.({sum([l[i,j,a].X for i in pos for j in pos])})")

Agent 0 have lunch.(0.0)
Agent 1 have lunch.(0.0)
Agent 2 have lunch.(0.0)
Agent 3 have lunch.(0.0)
Agent 4 have lunch.(0.0)
Agent 5 have lunch.(0.0)
Agent 6 have lunch.(0.0)
Agent 7 have lunch.(0.0)
Agent 8 have lunch.(0.0)
Agent 9 have lunch.(0.0)


In [22]:
LUNCH_BREAK_RANGE[0]

14400

In [23]:
for a in range(AGENTS):
    print("AGENT ",a)
    for i in pos:
        for j in pos:
            if y[i,j,a].X:
                print("___________________")
                print(data_dict[i]['SERVICE']+distance_matrix[i][j])
                print(s[i,a].X+data_dict[i]['SERVICE']+distance_matrix[i][j]+w[j,a].X)

AGENT  0
AGENT  1
AGENT  2
AGENT  3
AGENT  4
___________________
1846
4002.0
AGENT  5
___________________
1850
9046.0
___________________
1841
4312.0
___________________
1832
7196.0
AGENT  6
AGENT  7
___________________
3636
10077.0
___________________
1855
6300.0
___________________
1848
4445.0
AGENT  8
AGENT  9
___________________
1855
9740.0
___________________
1863
4319.0
___________________
1816
6189.0


In [24]:
for a in range(AGENTS):
    for i in pos:
        for j in pos:
            if y[i,j,a].X:
                print("------")
                print(s[i,a].X)
                print(s[j,a].X)

------
526.0
4002.0
------
7196.0
9046.0
------
939.0
4312.0
------
4312.0
7196.0
------
6300.0
10077.0
------
4445.0
6300.0
------
1523.0
4445.0
------
6189.0
9740.0
------
779.0
4319.0
------
4319.0
6189.0


In [25]:
for a in range(AGENTS):
    for i in pos:
        for j in pos:
            if y[i,j,a].X:
                print("------")
                print(c[i,j,a].X, "||", data_dict[j]['SERVICE'])

------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 3600
------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 1800
------
-0.0 || 1800
